In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
N = 1000

In [5]:
ip = np.random.randn(N)>0

In [6]:
#bpsk_modulation
s = 2 * ip - 1
s = np.array(s)


In [7]:
eb_no_db = np.arange(25)

In [8]:
bit_error_rate = np.zeros(len(eb_no_db)) 

In [11]:
def equal_gain_combining():
    
    bit_error_rate = np.zeros(len(eb_no_db))

    for jj in range(len(eb_no_db)):
        
        nErr = 0

        for ii in range(N):
            
            h11 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            h21 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            n11 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            n21 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            
            x = s[ii]
            
            H = np.array([[h11], [h21]])
            
            n = np.array([[n11], [n21]])
            
            y = np.dot(H, x) + n * (10 ** (-eb_no_db[jj] / 20))
            
            phase_compenstate = np.exp(-1j * np.angle(H))
            
            print(phase_compenstate)
            
            print(y)
            
            equalize_received_signal = np.dot(phase_compenstate,y)
            
            decoded_bits = (equalize_received_signal.real > 0).astype(int)

            nErr += np.sum(decoded_bits != ip)  # Accumulate errors for each iteration

        bit_error_rate[jj] = nErr / N  # Calculate bit error rate for the current SNR

    return bit_error_rate


In [12]:
result = equal_gain_combining()
print(result)
plt.figure()
plt.semilogy(eb_no_db, result, 'mo-', linewidth=2, markersize=8, label='nTx=2 nRx=2 (equal_gain)')

# plt.axis([0, 30, 1e-5, 0.1])
plt.grid(True)
plt.legend()
plt.xlabel('Eb/No, dB')
plt.ylabel('Bit Error Rate')
plt.title('BER for BPSK modulation with zero_forcing_algorithm')
plt.show()

[[0.40298765-0.91520542j]
 [0.59267377-0.80544261j]]
[[0.25801794-0.28168302j]
 [0.25520137-0.42662179j]]


ValueError: shapes (2,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)